In [1]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
import re 
import keras
from tensorflow.keras.layers import Input, TimeDistributed, GRU, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, Dense, Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D, Attention
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model, save_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
import random 
import time
from scipy import fftpack

In [2]:
train = pd.read_csv('train_features.csv') 
train_labels = pd.read_csv('train_labels.csv') 
test = pd.read_csv('test_features.csv') 
submission = pd.read_csv('sample_submission.csv')


In [3]:
X = tf.reshape(np.array(train.iloc[:,2:]), [-1,600,6]) 
X = np.asarray(X) 
X.shape

(3125, 600, 6)

In [4]:
y = train_labels['label'].values 
y.shape


(3125,)

In [5]:
def build_model():  
    inputs = Input(shape = (600,18)) 
    bn = BatchNormalization()(inputs)
    bi_gru = Bidirectional(GRU(128, return_sequences = True))(bn) 
    dropout = Dropout(0.25)(bi_gru) 
    bi_gru = Bidirectional(GRU(128, return_sequences = False))(dropout) 
    dense = Dense(128, activation = 'relu')(bi_gru) 
    bn = BatchNormalization()(dense) 
    outputs = Dense(61, activation = 'softmax')(bn)   
    model = Model(inputs = inputs, outputs = outputs) 
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) 
    return model

In [6]:
kfold = StratifiedKFold(n_splits = 10, random_state = 960418, shuffle = True)
for idx,(train_idx, val_idx) in enumerate(kfold.split(X,y)):   
    print("... Validating on fold {} ...".format(idx+1)) 
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx] 
    
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = [] 
    for i in tqdm(range(X_train.shape[0])): 
        for j in range(10): 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i]) 
    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)
    X_train = np.concatenate([X_train, X_augmented]) 
    y_train = np.concatenate([y_train, y_augmented])
    
    ###### feature engineering data ##### 
    print("... Feature Engineering ...")
    X_fourier_real = [] 
    X_fourier_imag = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
        real_part = np.fft.fft(X_train[i]).real 
        imag_part = np.fft.fft(X_train[i]).imag 
        X_fourier_real.append(real_part)
        X_fourier_imag.append(imag_part) 
    
    X_fourier_real = np.asarray(X_fourier_real)  
    X_fourier_imag = np.asarray(X_fourier_imag)
    X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)   
    
    
    X_val_fourier_real = [] 
    X_val_fourier_imag = [] 
    for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
        real_part = np.fft.fft(X_val[i]).real 
        imag_part = np.fft.fft(X_val[i]).imag 
        X_val_fourier_real.append(real_part) 
        X_val_fourier_imag.append(imag_part)
    
    X_val_fourier_real = np.asarray(X_val_fourier_real) 
    X_val_fourier_imag = np.asarray(X_val_fourier_imag)
    X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)
        
    
    ##### train model #####  
    print("... Training ...") 
    model = build_model() 
    model_path = 'kfold' + str(idx+1) + '/epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 7) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32,
              validation_data = (X_val, y_val),
              callbacks = [learning_rate_reduction, checkpoint, early_stopping])
    
    

 10%|█         | 285/2812 [00:00<00:00, 2846.07it/s]

... Validating on fold 1 ...
... Augmenting Data ...


  3%|▎         | 897/30932 [00:00<00:03, 8964.56it/s]

... Feature Engineering ...


100%|██████████| 313/313 [00:00<00:00, 8529.00it/s]


...


Epoch 00017: val_loss did not improve from 1.14152
... Validating on fold 10 ...
... Augmenting Data ...


  3%|▎         | 944/30943 [00:00<00:03, 9434.18it/s]

... Feature Engineering ...


100%|██████████| 312/312 [00:00<00:00, 9378.59it/s]


... Training ...
Epoch 1/200
967/967 [==============================] - 93s 91ms/step - loss: 3.2065 - accuracy: 0.3529 - val_loss: 1.9622 - val_accuracy: 0.5481

Epoch 00001: val_loss improved from inf to 1.96222, saving model to kfold10/epoch_001_val_1.962.h5
Epoch 2/200
967/967 [==============================] - 89s 92ms/step - loss: 1.8425 - accuracy: 0.5442 - val_loss: 1.2746 - val_accuracy: 0.6506

Epoch 00002: val_loss improved from 1.96222 to 1.27460, saving model to kfold10/epoch_002_val_1.275.h5
Epoch 3/200
967/967 [==============================] - 87s 90ms/step - loss: 1.2181 - accuracy: 0.6476 - val_loss: 1.1693 - val_accuracy: 0.6859

Epoch 00003: val_loss improved from 1.27460 to 1.16934, saving model to kfold10/epoch_003_val_1.169.h5
Epoch 4/200
967/967 [==============================] - 89s 92ms/step - loss: 0.8909 - accuracy: 0.7315 - val_loss: 1.0859 - val_accuracy: 0.7083

Epoch 00004: val_loss improved from 1.16934 to 1.08586, saving model to kfold10/epoch_004_val_

## Predict

In [7]:
gru1 = load_model('kfold1/epoch_006_val_0.972.h5')
gru2 = load_model('kfold2/epoch_004_val_1.197.h5') 
gru3 = load_model('kfold3/epoch_009_val_1.075.h5')  
gru4 = load_model('kfold4/epoch_007_val_0.969.h5') 
gru5 = load_model('kfold5/epoch_005_val_1.138.h5')
gru6 = load_model('kfold6/epoch_007_val_0.936.h5')
gru7 = load_model('kfold7/epoch_008_val_1.067.h5')
gru8 = load_model('kfold8/epoch_007_val_1.010.h5')
gru9 = load_model('kfold9/epoch_010_val_1.142.h5') 
gru10 = load_model('kfold10/epoch_004_val_1.086.h5') 


In [9]:
X_test = tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
X_test = np.asarray(test_X)

##### feature engineering for test dataset ##### 
X_test_fourier_real = [] 
X_test_fourier_imag = [] 
for i in tqdm(range(X_test.shape[0]), position = 0, leave = True):
    real_part = np.fft.fft(X_test[i]).real 
    imag_part = np.fft.fft(X_test[i]).imag 
    X_test_fourier_real.append(real_part) 
    X_test_fourier_imag.append(imag_part)
    
X_test_fourier_real = np.asarray(X_test_fourier_real) 
X_test_fourier_imag = np.asarray(X_test_fourier_imag)
X_test = np.concatenate([X_test, X_test_fourier_real, X_test_fourier_imag], axis = 2)  

print(X_test.shape)

100%|██████████| 782/782 [00:00<00:00, 9035.71it/s]

(782, 600, 18)


In [10]:
p1 = gru1.predict(X_test)

In [13]:
p2 = gru2.predict(X_test)
p3 = gru3.predict(X_test)
p4 = gru4.predict(X_test) 
p5 = gru5.predict(X_test) 
p6 = gru6.predict(X_test)
p7 = gru7.predict(X_test) 
p8 = gru8.predict(X_test) 
p9 = gru9.predict(X_test) 
p10 = gru10.predict(X_test)

In [14]:
p_avg = (p1 + p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9 + p10)/10.0 

In [16]:
submission.iloc[:,1:] = p_avg

In [17]:
submission.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000035,0.008070,1.592733e-05,0.000018,0.000013,4.770708e-08,0.352726,0.002920,0.000025,...,0.011902,5.910614e-04,0.000436,0.001321,1.436803e-06,0.000013,0.000198,0.122010,0.000005,0.000023
1,3126,0.000126,0.000812,5.921524e-06,0.021470,0.000004,6.673528e-05,0.000018,0.002538,0.000017,...,0.000009,6.659289e-07,0.000004,0.000250,3.787994e-05,0.000515,0.000336,0.000006,0.000032,0.000115
2,3127,0.000486,0.003486,6.210780e-06,0.000033,0.001839,1.618933e-04,0.070613,0.000983,0.000256,...,0.000582,3.588521e-03,0.000041,0.001242,8.133279e-06,0.000025,0.000115,0.019070,0.000449,0.015425
3,3128,0.000218,0.000011,4.616908e-05,0.000032,0.000003,3.350694e-06,0.000014,0.000017,0.000010,...,0.000118,3.847815e-05,0.000034,0.000061,5.054480e-05,0.000014,0.000958,0.000188,0.000006,0.003009
4,3129,0.049436,0.004557,1.857505e-07,0.000031,0.031574,1.093202e-05,0.000067,0.000006,0.000038,...,0.000013,1.700347e-06,0.000002,0.000001,5.227133e-07,0.000014,0.000324,0.000097,0.000037,0.000634


In [18]:
submission.to_csv("fourier_transform_10_fold_gru.csv",index=False)